## generate previous frames with pCO2 and fit it against ConvLSTM

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr
import sys
import tensorflow as tf
from tensorflow import keras
import os

sys.path.insert(0, '../../src')

from utils import df_to_xarray,read_xarray, custom_rmse

sys.path.insert(0, '../../src/preprocess')
from data_preprocess import preprocess_image_reduced,preprocess_images_nfp, inverse_scale_frame
from data_preprocess import preprocess_images, inverse_scale_image, preprocess_image_reduced

/burg/glab/users/sk4973/venv2/lib/python3.8/site-packages/xarray/backends/cfgrib_.py:27: UserWarning: Failed to load cfgrib - most likely there is a problem accessing the ecCodes library. Try `import cfgrib` to get the full error message
  warnings.warn(
2022-06-21 09:52:29.664004: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0


### Previous Frame generation

In [2]:
dist_map = pd.read_csv("../../src/dist_map.csv",header=None).to_numpy()
dist_map = np.roll(np.fliplr(dist_map),180)
dist_map = np.repeat(dist_map[np.newaxis, :, : ], 421, axis=0)


dir_name = "../../data/data1"
val_dir_name = "../../data/data2"

chl,mld,sss,sst,u10,xco2,icefrac,patm,pco2 = read_xarray(dir_name)


/burg/glab/users/sk4973/venv2/lib/python3.8/site-packages/xarray/backends/plugins.py:61: RuntimeWarning: Engine 'cfgrib' loading failed:
ecCodes library not found using ['eccodes', 'libeccodes.so', 'libeccodes']
  warnings.warn(f"Engine {name!r} loading failed:\n{ex}", RuntimeWarning)


In [3]:
# substituting mld with dist_map

chl_images = preprocess_image_reduced(chl.Chl.data)
sss_images = preprocess_image_reduced(sss.SSS.data)
sst_images = preprocess_image_reduced(sst.SST.data)
xco2_images = preprocess_image_reduced(xco2.XCO2.data,xco2=True)
y1 = preprocess_image_reduced(pco2.pCO2.data)
dist_map = preprocess_image_reduced(dist_map)
    

X1 = np.dstack((chl_images, dist_map, sss_images, sst_images, xco2_images))
X1 = X1.reshape((421,180,360,5),order='F')

INPUT_SHAPE=X1[0].shape
INPUT_SHAPE

(180, 360, 5)

In [4]:
chl,mld,sss,sst,u10,xco2,icefrac,patm,pco2 = read_xarray(val_dir_name, num = "035")

chl_images = preprocess_image_reduced(chl.Chl.data)
sss_images = preprocess_image_reduced(sss.SSS.data)
sst_images = preprocess_image_reduced(sst.SST.data)
xco2_images = preprocess_image_reduced(xco2.XCO2.data,xco2=True)
y2 = preprocess_image_reduced(pco2.pCO2.data)
dist_map = preprocess_image_reduced(dist_map)
    

X2 = np.dstack((chl_images, dist_map, sss_images, sst_images, xco2_images))
X2 = X2.reshape((421,180,360,5),order='F')

In [5]:
import sklearn
from sklearn.model_selection import train_test_split

X=np.concatenate((X1, X2), axis = 0)
y=np.concatenate((y1,y2),axis = 0)


train_X, val_X, train_y, val_y = train_test_split(X, y, test_size=0.3)

In [6]:
train_X.shape

(589, 180, 360, 5)

In [7]:
OUTPUT_SHAPE = y[0].shape

In [8]:
tf.keras.backend.clear_session()

from functools import partial

DefaultConv2D = partial(keras.layers.Conv2D,
                        kernel_size=5,activation='elu',
                        padding="SAME")

base_model = keras.models.Sequential([
    DefaultConv2D(filters=32, input_shape=INPUT_SHAPE),
    DefaultConv2D(filters=32),
    keras.layers.MaxPooling2D(pool_size=3),
    keras.layers.Dropout(0.2),
    DefaultConv2D(filters=64),
    DefaultConv2D(filters=64),
    keras.layers.MaxPooling2D(pool_size=3),
    keras.layers.Dropout(0.2),
    DefaultConv2D(filters=128),
    DefaultConv2D(filters=128),
    keras.layers.UpSampling2D(size=3),
    DefaultConv2D(filters=64),
    DefaultConv2D(filters=64),
    keras.layers.Dropout(0.2),
    keras.layers.UpSampling2D(size=3),
    DefaultConv2D(filters=32),
    DefaultConv2D(filters=2),
    DefaultConv2D(filters=1,kernel_size=1),
    keras.layers.Reshape(OUTPUT_SHAPE)
])

myLearnRate=0.0003
custom_opt = tf.keras.optimizers.Adam(learning_rate=myLearnRate)
base_model.compile(loss=custom_rmse, optimizer=custom_opt)

2022-06-21 09:52:42.320782: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-06-21 09:52:42.323470: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2022-06-21 09:52:42.362770: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:3b:00.0 name: Quadro RTX 8000 computeCapability: 7.5
coreClock: 1.62GHz coreCount: 72 deviceMemorySize: 44.49GiB deviceMemoryBandwidth: 581.23GiB/s
2022-06-21 09:52:42.362823: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
2022-06-21 09:52:42.411097: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.11
2022-06-21 09:52:42.411178: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublasLt.so.11
2022-0

InternalError: CUDA runtime implicit initialization on GPU:0 failed. Status: all CUDA-capable devices are busy or unavailable

In [ ]:
model_path="../../models/base_model/u_net_dist.h5"
early_stopings = tf.keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=10, verbose=1, mode='min')
checkpoint =  tf.keras.callbacks.ModelCheckpoint(model_path, monitor='val_loss', save_best_only=True, mode='min', verbose=0)
callbacks=[early_stopings,checkpoint]

history = base_model.fit(train_X,train_y, epochs=500, 
                         validation_data=(val_X,val_y),
                         workers=-1,batch_size=32,
                         callbacks=callbacks)



In [ ]:
cnn_model = tf.keras.models.load_model('../../models/base_model/u_net_dist.h5', custom_objects={'custom_rmse':custom_rmse})
predicted_image= cnn_model.predict(X1,verbose=1)
predicted_image[y1==0]=0.0

In [ ]:
pred_val= cnn_model.predict(X2,verbose=1)
pred_val[y2==0]=0.0

In [ ]:
print(custom_rmse(predicted_image,y1))
print(custom_rmse(predicted_image,y1))

In [ ]:
chl,mld,sss,sst,u10,xco2,icefrac,patm,pco2t2 = read_xarray(val_dir_name,num="035")

y_true,y_pred = inverse_scale_image(pred_val,pco2t2.pCO2.data)


y_pred_socat = np.copy(y_pred)
y_true_socat = np.nan_to_num(pco2t2.pCO2_socat.data)
y_pred_socat[y_true_socat==0]=0.0


print("val Full RMSE score:")
a=custom_rmse(y_pred,y_true)
print(a.numpy())

print("val SOCAT RMSE score:")
b=custom_rmse(y_pred_socat,y_true_socat)
print(b.numpy())

In [ ]:
chl,mld,sss,sst,u10,xco2,icefrac,patm,pco2t2 = read_xarray(dir_name)

y_true,y_pred = inverse_scale_image(predicted_image,pco2t2.pCO2.data)

y_pred_socat = np.copy(y_pred)
y_true_socat = np.nan_to_num(pco2t2.pCO2_socat.data)
y_pred_socat[y_true_socat==0]=0.0


print("Full RMSE score:")
a=custom_rmse(y_pred,y_true)
print(a.numpy())

print("SOCAT RMSE score:")
b=custom_rmse(y_pred_socat,y_true_socat)
print(b.numpy())

### Using the prediction as input in ConvLSTM

In [ ]:
chl,mld,sss,sst,u10,xco2,icefrac,patm,pco2 = read_xarray(dir_name)

chl_images = preprocess_image_reduced(chl.Chl.data)
sss_images = preprocess_image_reduced(sss.SSS.data)
sst_images = preprocess_image_reduced(sst.SST.data)
xco2_images = preprocess_image_reduced(xco2.XCO2.data,xco2=True)
y1 = preprocess_image_reduced(pco2.pCO2.data)
dist_map = preprocess_image_reduced(dist_map)
    

Xc1 = np.dstack((chl_images, dist_map, sss_images, sst_images, xco2_images,predicted_image))
Xc1 = Xc1.reshape((421,180,360,6),order='F')

In [ ]:
chl,mld,sss,sst,u10,xco2,icefrac,patm,pco2 = read_xarray(val_dir_name,num="035")

chl_images = preprocess_image_reduced(chl.Chl.data)
sss_images = preprocess_image_reduced(sss.SSS.data)
sst_images = preprocess_image_reduced(sst.SST.data)
xco2_images = preprocess_image_reduced(xco2.XCO2.data,xco2=True)
y2 = preprocess_image_reduced(pco2.pCO2.data)
dist_map = preprocess_image_reduced(dist_map)
    

Xc2 = np.dstack((chl_images, dist_map, sss_images, sst_images, xco2_images,pred_val))
Xc2 = Xc2.reshape((421,180,360,6),order='F')

In [ ]:
X=np.concatenate((Xc1, Xc2), axis = 0)
y=np.concatenate((y1,y2),axis = 0)

train_X, val_X, train_y, val_y = train_test_split(X, y, test_size=0.4)

In [ ]:
train_X_index=np.lib.stride_tricks.sliding_window_view(range(train_X.shape[0]),3) 
val_X_index=np.lib.stride_tricks.sliding_window_view(range(val_X.shape[0]),3) 

train_y=np.expand_dims(train_y[train_X_index][1:],axis=4)
train_X=train_X[train_X_index][:-1]

val_y=np.expand_dims(val_y[val_X_index][1:],axis=4)
val_X=val_X[val_X_index][:-1]

print("train set shape")
print(train_X.shape, train_y.shape)

print("val set shape")
print(val_X.shape, val_y.shape)

INPUT_SHAPE = train_X[0].shape


In [ ]:
tf.keras.backend.clear_session()

DefaultConvLSTM2D = partial(keras.layers.ConvLSTM2D,
                        filters=32, kernel_size=(5, 5),
                        padding="same",return_sequences=True,
                        activation="elu",)



model = keras.models.Sequential([
    DefaultConvLSTM2D(input_shape=INPUT_SHAPE),
    keras.layers.BatchNormalization(),
    DefaultConvLSTM2D(kernel_size=(3,3)),
    keras.layers.BatchNormalization(),
    DefaultConvLSTM2D(kernel_size=(1,1)),
    keras.layers.Conv3D(filters = 1, kernel_size=(3,3,3),activation="elu", padding="same")
    
])


model.compile(
    loss=custom_rmse, optimizer=keras.optimizers.Adam(learning_rate=0.0008),
)

In [ ]:
model_path="../../models/ConvLSTM_with_pco2.h5"

early_stopings = tf.keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=4, verbose=1, mode='min')
checkpoint =  tf.keras.callbacks.ModelCheckpoint(model_path, monitor='val_loss', save_best_only=True, mode='min', verbose=0)
callbacks=[early_stopings,checkpoint]

epochs = 50
batch_size = 8

# Fit the model to the training data.
hist = model.fit(
    train_X,
    train_y,
    batch_size=batch_size,
    epochs=epochs,
    validation_data=(val_X,val_y),
    callbacks=callbacks,
)

In [ ]:
X_index=np.lib.stride_tricks.sliding_window_view(range(421),3) 
#Xc1 = Xc1[X_index][:-1]
y1=np.expand_dims(y1[X_index][1:],axis=4)

best_model = tf.keras.models.load_model('../../models/ConvLSTM_with_pco2.h5', custom_objects={'custom_rmse':custom_rmse})
pred1=best_model.predict(Xc1,verbose=1)
pred1[y1==0]=0.0


chl,mld,sss,sst,u10,xco2,icefrac,patm,pco2t2 = read_xarray(dir_name)

y_true,y_pred = inverse_scale_frame(pred1,pco2t2.pCO2.data)

y_pred_socat = np.copy(y_pred)
y_true_socat=np.expand_dims(pco2t2.pCO2_socat.data[X_index][1:],axis=4)
y_true_socat = np.nan_to_num(y_true_socat)
y_pred_socat[y_true_socat==0]=0.0

In [ ]:
print("Full RMSE score:")
a=custom_rmse(y_pred[:,:2],y_true[:,:2])
print(a.numpy())

print("SOCAT RMSE score:")
b=custom_rmse(y_pred_socat[:,:2],y_true_socat[:,:2])
print(b.numpy())

In [ ]:
import imageio
import matplotlib.colors as mcolors


d = y_pred-y_true


norm = mcolors.TwoSlopeNorm(vmin=d.min(), vmax = d.max(), vcenter=0)


filenames = []

for i in range(418):
    # plot the line chart
    figure, axis = plt.subplots(2, 2,figsize=(12, 6))

    img=axis[0][0].imshow(np.flipud(y_pred[i][1]),cmap="coolwarm", interpolation="nearest")
    axis[0][0].set_title("prediction")
    plt.colorbar(img,ax=axis)

    img1=axis[0][1].imshow(np.flipud(y_true[i][1]),cmap="coolwarm", interpolation="nearest")
    axis[0][1].set_title("true")

    diff=np.flipud(np.squeeze(y_pred[i][1]-y_true[i][1]))
    img2=axis[1][0].imshow(diff,cmap="RdBu", interpolation="nearest",norm=norm)
    axis[1][0].set_title("residual")
    plt.colorbar(img2,ax=axis)
    
    img2=axis[1][1].imshow(np.flipud(Xc1[i][1][:,:,5]),cmap="coolwarm", interpolation="nearest")
    axis[1][1].set_title("input: previous predicted pco2")
    # create file name and append it to a list
    filename = f'{i}.png'
    filenames.append(filename)
    
    # save frame
    plt.savefig(filename)
    plt.close()


with imageio.get_writer('../../assets/cnn-lstm-new.gif', mode='I') as writer:
    for filename in filenames:
        image = imageio.imread(filename)
        writer.append_data(image)
        
# Remove files
for filename in set(filenames):
    os.remove(filename)